In [1]:
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# 1. read in data

In [2]:
def get_mutated_sequence(mut, sequence_wt):
  wt, pos, mt = mut[0], int(mut[1:-1]), mut[-1]

  sequence = deepcopy(sequence_wt)

  return sequence[:pos]+mt+sequence[pos+1:]

def get_pos_sequence(mut):
  wt, pos, mt = mut[0], int(mut[1:-1]), mut[-1]

  return pos

In [3]:
with open('Hackathon_data/sequence.fasta', 'r') as f:
  data = f.readlines()

sequence_wt = data[1].strip()

In [4]:
import numpy as np
from copy import deepcopy

# read in all train data
df_base = pd.read_csv('Hackathon_data/train.csv')
df_base['mut_seq'] = df_base.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_base['pos'] = df_base.mutant.apply(lambda x: get_pos_sequence(x))
df_base['wt_seq'] = sequence_wt
df_base.loc[:, "score"] = df_base["DMS_score"]

df_train2 = pd.read_csv('Hackathon_data/train2.csv')
df_train2['mut_seq'] = df_train2.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_train2['pos'] = df_train2.mutant.apply(lambda x: get_pos_sequence(x))
df_train2['wt_seq'] = sequence_wt
df_train2.loc[:, "score"] = df_train2["DMS_score"]

df_train3 = pd.read_csv('Hackathon_data/train3.csv')
df_train3['mut_seq'] = df_train3.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_train3['pos'] = df_train3.mutant.apply(lambda x: get_pos_sequence(x))
df_train3['wt_seq'] = sequence_wt
df_train3.loc[:, "score"] = df_train3["DMS_score"]

df_train4 = pd.read_csv('Hackathon_data/train4.csv')
df_train4['mut_seq'] = df_train4.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_train4['pos'] = df_train4.mutant.apply(lambda x: get_pos_sequence(x))
df_train4['wt_seq'] = sequence_wt
df_train4.loc[:, "score"] = df_train4["DMS_score"]

In [5]:
# combine initial train data and query data
df_train = pd.concat([df_base, df_train2, df_train3, df_train4], ignore_index=True)
df_train

mutant  DMS_score                                            mut_seq  \
0       M0Y   0.273000  YVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
1       M0W   0.285700  WVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
2       M0V   0.215300  VVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
3       M0T   0.312200  TVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
4       M0S   0.218000  SVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
...     ...        ...                                                ...   
1435  F581E   0.855451  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
1436  A586P   0.989355  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
1437  R593P   0.806492  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
1438  N619I   0.911438  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   
1439  V634P   0.569993  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...   

      pos                                             wt_seq     score  \
0       0  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.273000   
1       0  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.285700   
2       0  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.215300   
3       0  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.312200   
4       0  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.218000   
...   ...                                                ...       ...   
1435  581  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.855451   
1436  586  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.989355   
1437  593  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.806492   
1438  619  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.911438   
1439  634  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  0.569993   

                                               sequence  
0                                                   NaN  
1                                                   NaN  
2                                                   NaN  
3                                                   NaN  
4                                                   NaN  
...                                                 ...  
1435  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  
1436  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  
1437  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  
1438  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  
1439  MVNEARGNSSLNPCLEGSASSGSESSKDSSRCSTPGLDPERHERLR...  

[1440 rows x 7 columns]

In [6]:
# read in test data
df_test = pd.read_csv('Hackathon_data/test.csv')
df_test['mut_seq'] = df_test.mutant.apply(lambda x: get_mutated_sequence(x, sequence_wt))
df_test['pos'] = df_test.mutant.apply(lambda x: get_pos_sequence(x))
df_test['wt_seq'] = sequence_wt
df_test.loc[:, "score"] = 0

# 2. Get AA Features

In [7]:
from aaindex import aaindex1

# Get all 20 standard amino acids
aa_list = list("ACDEFGHIKLMNPQRSTVWY")

# Collect all usable AAindex1 codes (those with all 20 AAs)
record_codes = [
    code for code in aaindex1.record_codes()
    if all(aa in aaindex1[code].values for aa in aa_list)
]

print(f"Using {len(record_codes)} AAindex records.")

# Build the embedding matrix: 20 AA × N features
aa_matrix = np.array([
    [aaindex1[code].values[aa] for code in record_codes]
    for aa in aa_list
], dtype=np.float32)  # shape: (20, num_records)

# normalize across features (z-score normalization)
aa_matrix = (aa_matrix - aa_matrix.mean(axis=0)) / aa_matrix.std(axis=0)

# Build the final embedding dictionary
aaindex_dict = {
    aa: aa_matrix[i].tolist()
    for i, aa in enumerate(aa_list)
}

Using 566 AAindex records.


In [8]:
def get_AA_embeddings(mut_str, aaindex_dict, length=656):
    ref_aa = mut_str[0]
    mut_aa = mut_str[-1]
    pos = int(mut_str[1:-1])

    ref_emb = np.array(aaindex_dict.get(ref_aa))
    mut_emb = np.array(aaindex_dict.get(mut_aa))

    position = pos / length
    region = np.zeros(5)
    region_idx = min(pos * 5 // length, 4)  # ensure index stays in 0-4
    region[region_idx] = 1.0



    full_feature = np.concatenate([
        mut_emb - ref_emb, [position], region
    ])
    return full_feature

In [9]:
# form AA feature matrix for train data
features = df_train.apply(
    lambda row: get_AA_embeddings(row['mutant'], aaindex_dict),
    axis=1
)
XAA = np.stack(features.values)
y = df_train['DMS_score'].values

In [10]:
# form AA feature matrix for test data
features_test = df_test.apply(
    lambda row: get_AA_embeddings(row['mutant'], aaindex_dict),
    axis=1
)
X_test = np.stack(features_test.values)

# 3 Grid Search

In [11]:
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import spearmanr
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer
from sklearn.gaussian_process.kernels import RBF, DotProduct, RationalQuadratic, ExpSineSquared, WhiteKernel
def spearman_scorer(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation
custom_scorer = make_scorer(spearman_scorer, greater_is_better=True)

In [12]:
models = {
    'Ridge':Ridge(),
    'GradientBoostingRegressor':GradientBoostingRegressor(),
    'XGBRegressor': XGBRegressor(random_state=42),
    'RandomForestRegressor': RandomForestRegressor(random_state=42),
    'KNN': KNeighborsRegressor(),
    'SVR': SVR()
}


param_grids = {
    'Ridge':{
        'alpha': [0.01, 0.1, 1.0, 10.0, 100]
    },
    'GradientBoostingRegressor':{
        'n_estimators':[50, 100, 200],
        'max_depth':[3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample':[0.8, 1.0]
    },
    'XGBRegressor': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0],
    },
    'RandomForestRegressor': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, None],
        'min_samples_leaf': [1, 5, 10],
        'min_samples_split': [2, 10, 20],
        'bootstrap': [True, False],
    },
    'KNN':{
        'n_neighbors':[5,10],
        'weights':['uniform', 'distance'],
        'algorithm':['ball_tree', 'kd_tree', 'brute'],
        'leaf_size':[15, 30],
        'p':[1,2]
    },
    'SVR': {
        'C': [0.1, 0.5, 1.0, 10],
        'gamma': ['scale', 'auto'],
        'epsilon': [0.05, 0.1, 0.2],
        'kernel': ['rbf', 'linear', 'poly']
    }
}

In [ ]:

# Performa grid search for a list of models
n_splits = 5
seed = 42
gkf = GroupKFold(n_splits=n_splits)
group = df_train['pos']

# --- Main loop ---
for model_name, model in models.items():
    print(f"\n--- Grid Search: {model_name.upper()} ---")
    
    grid = GridSearchCV(
        estimator = model,
        param_grid= param_grids[model_name],
        scoring= custom_scorer,
        cv=gkf,
        n_jobs=-1,
        verbose=0
    )

    grid.fit(XAA, y, groups = group)
    best_model = grid.best_estimator_
    best_params = grid.best_params_
    
    print(best_params)
    
    spearman_scores = []

    gkf = GroupKFold(n_splits=n_splits)
    for fold, (train_idx, val_idx) in enumerate(gkf.split(XAA, y, groups=df_train['pos'])):
        X_train, X_val = XAA[train_idx], XAA[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Train & predict
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_val)

        # Spearman correlation
        corr, _ = spearmanr(y_val, y_pred)
        spearman_scores.append(corr)
        print(f"Fold {fold + 1}: Spearman = {corr:.4f}")

    # Final results
    avg_corr = np.mean(spearman_scores)
    std_corr = np.std(spearman_scores)
    print(f"Avg Spearman: {avg_corr:.4f} ± {std_corr:.4f}")


--- Grid Search: RIDGE ---
{'alpha': 1.0}
Fold 1: Spearman = 0.4385
Fold 2: Spearman = 0.4552
Fold 3: Spearman = 0.3862
Fold 4: Spearman = 0.2931
Fold 5: Spearman = 0.3278
Avg Spearman: 0.3801 ± 0.0623

--- Grid Search: GRADIENTBOOSTINGREGRESSOR ---
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Fold 1: Spearman = 0.5388
Fold 2: Spearman = 0.5889
Fold 3: Spearman = 0.4792
Fold 4: Spearman = 0.3762
Fold 5: Spearman = 0.4192
Avg Spearman: 0.4805 ± 0.0772

--- Grid Search: XGBREGRESSOR ---
{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}
Fold 1: Spearman = 0.5384
Fold 2: Spearman = 0.6023
Fold 3: Spearman = 0.5118
Fold 4: Spearman = 0.4695
Fold 5: Spearman = 0.4226
Avg Spearman: 0.5089 ± 0.0610

--- Grid Search: RANDOMFORESTREGRESSOR ---
{'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 200}
Fold 1: Spearman = 0.5360
Fold 2: Spearman = 0.6096
Fold 3: Spearman = 0.4945
Fold 4: S

# Train Ensemble Model

In [12]:
# Choose a set of model based on grid search (some modification was made later after adding regularization parameters)
model1 = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.01,
    subsample=0.8,
    min_samples_leaf=10,
    min_samples_split=20,
    random_state=0
)

model2 =  XGBRegressor(
            n_estimators=200,
            max_depth=3,
            learning_rate=0.01,                         
            reg_lambda=1.0,             
            reg_alpha=0.0, 
            subsample=0.8,
            n_jobs=-1,
            random_state = 1
)

model3 = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_leaf=10,
    min_samples_split=20,
    max_features='sqrt',
    bootstrap=True,
    random_state=2
)

model4 = SVR(
    kernel='rbf',
    gamma='scale',
    epsilon=0.1,
    C=0.5,
)
emsembled_models = [model1, model2, model3, model4]

In [13]:
# Train the ensemble model
trained_models = []

for i, model in enumerate(emsembled_models):

    model.fit(XAA, y)
    y_pred = model.predict(XAA)
    spearman_corr, _ = spearmanr(y, y_pred)
    trained_models.append(model)

In [14]:
# Make prediction on test set
preds = []

for i, model in enumerate(trained_models):
    pred = model.predict(X_test)
    preds.append(pred)
    
preds = np.stack(preds, axis=0)
preds_mean = np.mean(preds, axis=0)
df_test['DMS_score_predicted'] = preds_mean

In [15]:
# Output prediction results
df_test[['mutant', 'DMS_score_predicted']].to_csv('Ensemble_Model_Predictions.csv',index=False)

# (Optional) Make new queries

In [ ]:
preds_std = np.std(preds, axis=0)

In [ ]:
# we can use uncertainty sampling for new queries
top100_indices = np.argsort(preds_std)[-100:]
mutants = [df_test.iloc[i]['mutant'] for i in topk_indices]

# Write to a .txt file
with open('query.txt', 'w') as f:
    for mutant in mutants:
        f.write(mutant+'\n')